If we are running our jupyter server through docker, our webcam might not be seen by our guest environment. That is, the container running jupyter has no devices.

That can be changed with the --device option when spinning up new conainers.

In [3]:
# In the host environment we need to first find the device

# $ ls /dev | grep video
#    > video0
#      video1

# $ ls /dev | grep video
#    > video0
#      video1

In [4]:
# Then we just need to connect the device to the container.

In [ ]:
#For instance, if we previously did this:

docker run -it -p 8888:8888 -v `pwd`:/root/jupyterdata pjsousa/docker-jupyteropencv

In [ ]:
#it becomes this:

docker run -it --device /dev/video0 -p 8888:8888 -v `pwd`:/root/jupyterdata pjsousa/docker-jupyteropencv
        
docker run -it --device /dev/video0 -p 8888:8888 -v `pwd`:/root/jupyterdata pjsousa/docker-jupyteropencv

Now, given we're running jupyter inside docker, we won't be able to see our webcam in the window that OpenCV usually opens for us.

One (really slow) way of looking at the output of the webcam is to render the images inline into the jupyter's cell output

In [13]:
# first, we are already overriden the cv2.imshow with something similiar to this

#def imshow(cv2, name, input, fig_size=None, cmap="gray"):
#    if fig_size is None:
#        dpi = 80
#        im_data = input
#        try:
#            height, width, depth = im_data.shape
#        except:
#            height, width = im_data.shape
#        fig_size = width / float(dpi), height / float(dpi)

#    try:
#        input = cv2.cvtColor(input,cv2.COLOR_BGR2RGB)
#    except :
#        input = input

#    import matplotlib.pyplot as plt
#    get_ipython().magic(u'matplotlib inline')
    
#    fig, ax = plt.subplots(figsize=fig_size)
#    print("Showing Image: {}".format(name))
#    ax.imshow(input, interpolation='none', aspect="auto", cmap=cmap)
#    plt.show()

# a version of this function is also in this gist:
#     https://gist.github.com/pjsousa/5dd693491c3ee7a4879ca06249e96d2a

Now we just need to fetch images from our device and show them inside the notebook. 

In [ ]:
import notebook_util
import cv2
from matplotlib import pyplot as plt
import numpy as np
import time
from IPython.display import clear_output

notebook_util.override_imshow(cv2) # this overrides cv2.imshow

%matplotlib inline

In [ ]:
cap = cv2.VideoCapture(0) ## we want the video device 0 (we probably only have one inside the container...)

try:
    while True:
        # we'll clear the output cell;
        clear_output()
        
        #get a new frame;
        ret, frame = cap.read()
        
        #And show the frame's image
        cv2.imshow('Our Live, slow, video ', frame)
        
        # wait for a second, so the notebook doesn't hang
        time.sleep(1)

except KeyboardInterrupt:
    # Release camera
    cap.release()

In order to stop this example we need to actually send the KeyboardInterrupt signal. Which must be handled and used to stop the device, otherwise the camera would be locked.